In [20]:
import pandas as pd

fs = pd.read_csv("../data/raw/FS.csv")

fs.drop(columns=[
     '자산(*)(IFRS연결)(천원)',
 '자본(*)(IFRS연결)(천원)',
 '부채(*)(IFRS연결)(천원)',
 '유동자산(*)(IFRS연결)(천원)',
 '유동부채(*)(IFRS연결)(천원)',
 '* 발행한 주식총수(*)(IFRS연결)(천원)',
 '자본금(*)(IFRS연결)(천원)',
 '이익잉여금(결손금)(*)(IFRS연결)(천원)',
 '매출액(수익)(*)(IFRS연결)(천원)',
 '* (정상)영업손익(보고서기재)(IFRS연결)(천원)',
], inplace=True)

df = pd.read_csv("../data/raw/당기순이익.csv")
profit = "당기순이익(손실)(IFRS)(천원)"

# fs와 df의 원본 데이터 비교 후 merge 또는 concat 수행
fs_raw_cols = ['회사명', '거래소코드', '회계년도']
df_raw_cols = ['회사명', '거래소코드', '회계년도']

# 공통 컬럼들이 같은지 확인 후 완벽한 일치를 위한 merge 수행
if all(col in fs.columns for col in fs_raw_cols) and all(col in df.columns for col in df_raw_cols):
    # merge를 사용하여 완벽한 일치 보장
    result = pd.merge(fs, df[['회사명', '거래소코드', '회계년도', profit]], 
                     on=['회사명', '거래소코드', '회계년도'], how='inner')
    
    # merge 후 원본 df와 같은 순서로 정렬
    result = result.reset_index(drop=True)
    
    # 길이가 다른 경우에만 알림
    if len(df) != len(result):
        print(f"⚠️ 길이 차이 발생: 원본 df({len(df)}) vs 결과 result({len(result)})")
        print("이는 merge 과정에서 일치하지 않는 키가 제거되었기 때문입니다.")
else:
    # 컬럼이 없다면 단순 concat (인덱스 기반)
    result = pd.concat([fs, df[[profit]]], axis=1)
    result = result.reset_index(drop=True)
    
    if len(df) != len(result):
        print(f"⚠️ 길이 차이 발생: 원본 df({len(df)}) vs 결과 result({len(result)})")



⚠️ 길이 차이 발생: 원본 df(23813) vs 결과 result(23833)
이는 merge 과정에서 일치하지 않는 키가 제거되었기 때문입니다.
